## Import Packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.datasets import make_classification
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# Problem Understanding

The Vehicle Safety Board of Chicago has determine that it is necessary to know and understand the primary contributory cause of a car accident. Therefore, it will be possible to recognise patterns and consequently "it could create guidelines to implement preventive actions."
Based on the data, we are going to predict if the cause of a car accident if influence by the weather, location, time or by the number of passengers.
From the created models, we want to estimate the global/generic/general/average cost of having a injury person or the cost a average crash.


Predict the frequency of traffic accidents between the road geometric variables, traffic characteristics, and environmental factors.


# Data Understanding



In [ ]:
# Crashes Dataset
crash = pd.read_csv('data/Traffic_Crashes_-_Crashes.csv')
# crash.info()

In [ ]:
# People Dataset
people = pd.read_csv('data/Traffic_Crashes_-_People.csv')
# people.head()

In [ ]:
# Vehicles Dataset
vehicles = pd.read_csv('data/Traffic_Crashes_-_Vehicles.csv')
# crashes.head()

## Selecting Features

In [ ]:
# Selecting specific features on crashes dataset
crashes = crash[['CRASH_RECORD_ID', 'POSTED_SPEED_LIMIT', 'WEATHER_CONDITION', 
                 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'ROADWAY_SURFACE_COND',
                 'ROAD_DEFECT', 'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE',
                 'BEAT_OF_OCCURRENCE','CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH']]

In [ ]:
# Selecting features on people's dataset
peoples = people[['CRASH_RECORD_ID','PERSON_TYPE', 'SEX', 'AGE']]

In [ ]:
# Selecting features on vehicle's dataset
vehicle = vehicles[['CRASH_RECORD_ID', 'VEHICLE_TYPE', 'MANEUVER']]

## Combining Datasets

In [ ]:
#  Merging crashes with people datastes
df_cp = pd.merge(crashes, peoples, on = 'CRASH_RECORD_ID', how = 'left')

In [ ]:
# Merging df_cp with vehicles dataset
df_x = pd.merge(df_cp, vehicle, on = 'CRASH_RECORD_ID', how = 'left')

In [ ]:
# Selecting only class Driver
df_t = df_x[(df_x.PERSON_TYPE == 'DRIVER') & (df_x.AGE >=16) & (df_x.AGE <= 100)]
# df_t.isnull().sum()

In [ ]:
dfq = df_t.drop_duplicates(subset = 'CRASH_RECORD_ID', keep='first')
dfq.isnull().sum()

In [ ]:
# Dropping all nun values
df = dfq.dropna()
df.reset_index(inplace=True)

In [ ]:
# Dropping all nun values
df = dfq.dropna()
df.reset_index(inplace=True)

## Data Analysis

In [ ]:
# Geographic distribution of the crashes
from sodapy import Socrata
from shapely.geometry import shape

end_point = 'data.cityofchicago.org'
app_token = 'IaMIsAtmEfVIRb6i2lfqYvVmz'
max_rows = 300000

client = Socrata(end_point, app_token)
chi_map = client.get("85ca-t3if", limit=max_rows)
chi_map = pd.DataFrame(chi_map)

In [ ]:
# Converting Latitude and Longitude as float
chi_map['latitude'] = chi_map['latitude'].astype(float)
chi_map['longitude'] = chi_map['longitude'].astype(float)
chi_map = chi_map[np.abs(chi_map['latitude'] - chi_map['latitude'].mean()) <= (10 * chi_map['latitude'].std())]
chi_map = chi_map[np.abs(chi_map['longitude'] - chi_map['longitude'].mean()) <= (10 * chi_map['longitude'].std())]

In [ ]:
import geopandas as gpd
chi_map['location'] = chi_map['location'].apply(shape)
crs = {'init': 'epsg:4326'} 
chi_map = gpd.GeoDataFrame(chi_map, geometry='location', crs=crs)

chi_map.shape

In [ ]:
chi_map.plot(markersize=0.01, color='green', figsize=(12,12))
plt.axis('on')
plt.title('Crashes Geographically Distributed - Chicago')
plt.show()

In [ ]:
zipfile = "zip://data/beats_boundaries.zip"
beats_map = gpd.read_file(zipfile)

In [ ]:
beats_map.rename(columns={'beat_num': 'BEAT_OF_OCCURRENCE'}, inplace=True)
beats_map['BEAT_OF_OCCURRENCE'] = beats_map['BEAT_OF_OCCURRENCE'].astype(int)

In [ ]:
x = pd.DataFrame(crashes['BEAT_OF_OCCURRENCE'].value_counts())
x['index'] = x.index
# ignore the error
beats_index = x.merge([x,x['index']])

In [ ]:
x.reset_index(inplace=True)
x.drop(columns='index', inplace=True)
x.rename(columns={"level_0": "BEAT_OF_OCCURRENCE", "BEAT_OF_OCCURRENCE": "COUNT"}, inplace=True)
x['BEAT_OF_OCCURRENCE'] = x['BEAT_OF_OCCURRENCE'].astype(int)


In [ ]:
merged = beats_map.set_index('BEAT_OF_OCCURRENCE').join(x.set_index('BEAT_OF_OCCURRENCE'))
merged.fillna(0, inplace=True)
merged['COUNT'] = merged['COUNT'].astype(int)

In [ ]:
fig, ax = plt.subplots(1, figsize=(20, 16))
ax = merged.plot(ax=ax, 
              column='COUNT', 
                 cmap='OrRd',
                 alpha = .5, 
                 linewidth=.5, 
                 edgecolor='black',  
                 legend = True,
                 vmin=40,
                 vmax=4000)
ax.set_title('Concentration of car crashes in Chicago', fontsize = 20)
ax.set_axis_off()
fig.tight_layout
plt.show();

In [ ]:

crash['CRASH_TYPE'].value_counts().plot(kind='bar', 
                                        title ="CRASH_TYPE",
                                        figsize=(10, 5),
                                        legend=True, 
                                        fontsize=12,
                                        color='slategrey')
plt.show()

In [ ]:

crash['MOST_SEVERE_INJURY'].value_counts().plot(kind='bar',
                                                title ="MOST_SEVERE_INJURY", 
                                                figsize=(10, 5), 
                                                legend=True,
                                                fontsize=12,
                                                color='slategrey')
plt.show()

In [ ]:

crash['DAMAGE'].value_counts().plot(kind='bar', 
                                    title ="DAMAGE", 
                                    figsize=(10, 5),
                                    legend=True,
                                    fontsize=12,
                                    color='slategrey')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,10))
sns.set(style="darkgrid")
ax = sns.countplot(x="CRASH_DAY_OF_WEEK", data=crashes, color='navy')
plt.title('Crash based on the day of the week')
plt.xlabel('Day (1 = Sunday)', size=15)
plt.ylabel('Crashes', size=15, rotation=0);

In [ ]:
fig = plt.figure(figsize=(10,10))
sns.set(style="darkgrid")
ax = sns.countplot(x="CRASH_HOUR", data=crashes, color='firebrick')
plt.title('Crash based on the hour of day')
plt.xlabel('Hour (24H)', size=15)
plt.ylabel('Crashes', size=15, rotation=0);


In [ ]:
crash['STREET_NAME'].value_counts()[:min(20, len(crash))].plot(kind='bar',
                                                               title ="CRASH BY STREET", 
                                                               figsize=(10, 5),
                                                               legend=True,
                                                               fontsize=12,
                                                               color='slategrey')
plt.show()

In [ ]:

crash['WEATHER_CONDITION'].value_counts().plot(kind='bar',
                                               title ="WEATHER_CONDITION",
                                               figsize=(10, 5),
                                               legend=True,
                                               fontsize=12,
                                               color='slategrey')
plt.show()

# Data Preparation

In [ ]:
# Creating dummy variables 
x_feats = ['WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE',
           'ROADWAY_SURFACE_COND','ROAD_DEFECT','VEHICLE_TYPE', 'MANEUVER',
           'SEX', 'SEC_CONTRIBUTORY_CAUSE','PERSON_TYPE']
    
tre = pd.get_dummies(df[x_feats], drop_first=True, dtype=float)
tre

In [ ]:
y_dummy = ['PRIM_CONTRIBUTORY_CAUSE']
y_dum = pd.get_dummies(df[y_dummy], drop_first=True, dtype=float)

In [ ]:
y_dum['PRIM_CONTRIBUTORY_CAUSE_FOLLOWING TOO CLOSELY'].value_counts()

In [ ]:
qw = df[['BEAT_OF_OCCURRENCE', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK']]

### Split your X data in train and test datasets¶


In [ ]:
X = pd.concat([qw, tre], axis=1)
y = y_dum['PRIM_CONTRIBUTORY_CAUSE_FOLLOWING TOO CLOSELY']

In [ ]:
# Split X data in train and test datasets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split train data in train and validation datasets
X_train_s, X_val, y_train_s, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print(len(X_train_s), len(X_val), len(y_train_s), len(y_val))

In [ ]:
print('X_train_s:', X_train_s.shape)
print('y_train_s:', y_train_s.shape)
print('X_val:', X_val.shape)
print('y_val:', y_val.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

### Scale the 3 datasets using StandardScaler¶


In [ ]:
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler()
scaler.fit(X_train_s)

X_train_scaled = scaler.transform(X_train_s)
X_val_scaled = scaler.transform(X_val) 
X_test = scaler.transform(X_test)

# Model Evaluation

### Baseline Model

In [ ]:
# Instantiate and fit a DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=42) 
tree_clf.fit(X_train_scaled,y_train_s)

In [ ]:
tr_p_t = tree_clf.predict_proba(X_train_scaled)[:,1]
tr_pr_v = tree_clf.predict_proba(X_val_scaled)[:,1]

In [ ]:
print("ROC AUC for the train dataset is", round(roc_auc_score(y_train_s, tr_p_t), 4))
print("ROC AUC for the validation dataset is", round(roc_auc_score(y_val, tr_pr_v), 4))

## Bagging Classifier

In [ ]:
from sklearn.ensemble import BaggingClassifier
bagged_tree = BaggingClassifier(DecisionTreeClassifier(criterion='gini', max_depth=5), n_estimators=20, random_state=42)
bagged_tree.fit(X_train_scaled, y_train_s)

In [ ]:
bag_predi_t = bagged_tree.predict_proba(X_train_scaled)[:,1]
bag_predi_v = bagged_tree.predict_proba(X_val_scaled)[:,1]

In [ ]:
print("ROC AUC for the train dataset is", round(roc_auc_score(y_train_s, bag_predi_t), 4))
print("ROC AUC for the validation dataset is", round(roc_auc_score(y_val, bag_predi_v), 4))

### Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
skf = StratifiedKFold(n_splits=5, random_state=42)

param_grid = {'n_estimators' : [20,30],
              'bootstrap_features' : [True, False],
              'oob_score' : [True, False],
              }

opt_model = GridSearchCV(bagged_tree,
                         param_grid,
                         cv=skf,
                         scoring='accuracy',
                         return_train_score=True)

opt_model.fit(X_train_scaled, y_train_s)

print('The best hyperparameters are:',opt_model.best_params_)
print("Best score of the model is", round(opt_model.best_score_,4))

In [ ]:
best_model = opt_model.best_estimator_
best_model

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100,n_jobs=-1, random_state=42)
rfc.fit(X_train_scaled,y_train_s)


In [ ]:
rfc_p_t = rfc.predict_proba(X_train_scaled)[:,1]
rfc_p_val = rfc.predict_proba(X_val_scaled)[:,1]

In [ ]:
print("ROC AUC for the train dataset is", round(roc_auc_score(y_train_s, rfc_p_t), 4))
print("ROC AUC for the validation dataset is", round(roc_auc_score(y_val, rfc_p_val), 4))

### Hyperparameter 

In [ ]:
rfc_param_grid = {'max_depth':range(1,11,50),
                  'min_samples_leaf':[5,10,15]}

opt_rfc = GridSearchCV(rfc, 
                       rfc_param_grid,
                       cv=skf, 
                       scoring='accuracy',
                       n_jobs=-1)

opt_rfc.fit(X_train_scaled, y_train_s)

print('The best hyperparameters are:',opt_rfc.best_params_)
print("Best score of the model is", round(opt_rfc.best_score_,4))

In [ ]:
opt_rfc.fit(X_train_scaled, y_train_s)
print('Values of the optimised hyperparameters\nfor the best model found:',
      opt_rfc.best_params_)
round(opt_rfc.best_score_, 3)

In [ ]:
best_model = opt_rfc.best_estimator_
best_model

In [ ]:
from sklearn.metrics import roc_curve
fpr_t,tpr_t,thr_t = roc_curve(y_train_s,bag_predi_t)
fpr_v,tpr_v,thr_v = roc_curve(y_val,bag_predi_v)

plt.title('ROC Validation')
plt.plot(fpr_t, tpr_t, 'red', label = 'AUC')
plt.plot(fpr_v, tpr_v, 'b', label = 'AUC val')
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
CFP = 650
TNC = 0
FNC = 47921
TPC = 20728
prevalence = .025
m = ((1 - prevalence) / prevalence ) * ((CFP - TNC) / (FNC - TPC))

print( f' m coefficient is {round(m,2)}')

# FP cost is the mean of the ambulance dispatch with and without health insurance
# https://health.costhelper.com/ambulance.html

# FN costs are a mean of all possible car injuries levels.
# Fatalities are excluded given the almost absence from our target crash type.
# extracted from the KABCO table, page 15
# https://safety.fhwa.dot.gov/hsip/docs/fhwasa17071.pdf

# TP costs is the sum of the means of medical bills and crash repairs.
# extracted from a Verisk Analysis report from 2013, prices adjusted for inflation

In [ ]:

fpr, tpr, threshold = metrics.roc_curve(y_train_s,bag_predi_t)
roc_auc = metrics.auc(fpr, tpr)

In [ ]:
# finding TPR and FPR for threshold 0.4
thresh_40 = threshold[65]
fpr_40 = fpr[65]
tpr_40 = tpr[65]

fm_40 = tpr_40 - m*fpr_40

print(f' The fm for the .4 threshold is {round(fm_40,2)}')

# finding TPR and FPR for threshold 0.5
thresh_50 = threshold[46]
fpr_50 = fpr[46]
tpr_50 = tpr[46]

fm_50 = tpr_50 - m*fpr_50

print(f' The fm for the .5 threshold is {round(fm_50,2)}')

# finding TPR and FPR for threshold 0.6
thresh_60 = threshold[33]
fpr_60 = fpr[33]
tpr_60 = tpr[33]

fm_60 = tpr_60 - m*fpr_60

print(f' The fm for the .6 threshold is {round(fm_60,2)}')

# finding TPR and FPR for threshold 0.7
thresh_70 = threshold[25]
fpr_70 = fpr[25]
tpr_70 = tpr[25]

fm_70 = tpr_70 - m*fpr_70

print(f' The fm for the .7 threshold is {round(fm_70,2)}')

In [ ]:

# creating the confusion matrix with sklearn
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_s, bag_predi_t)